# Geez Number Prediction

In this project will built and training Geez number prediction model. We will use a dataset consisting of Geez/ Amharic digits (1-9). We will  develop a deep, fully-connected ("feed-forward") neural network model that can classify these images. In the process we will discuss:<br>

* Data collection and preparation
* Implementing Deep feed forward Neural networks
  - Libraries
  - Reading Data with data generators
      - Data augmentation
* Transfer Learning

In [ ]:
import keras
from keras_preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, Activation
from keras.optimizers  import RMSprop, SGD, Adam
from keras.losses import sparse_categorical_crossentropy
import matplotlib.pyplot as plt
import numpy as np
import os
from keras import models
import cv2

## Data Generators

**Motivation**

Data preparation is required when working with neural network and deep learning models. Increasingly data augmentation is also required on more complex object recognition tasks.

Like the rest of Keras, the image augmentation API is simple and powerful.

Keras provides the *ImageDataGenerator* class that defines the configuration for image data preparation and augmentation. This includes capabilities such as:
* Sample-wise standardization.
* Feature-wise standardization.
* ZCA whitening.
* Random rotation, shifts, shear and flips.
* Dimension reordering.
* Save augmented images to disk.